In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
data_DoS = pd.read_csv('/content/drive/MyDrive/ML/botiot/UNSW_2018_IoT_Botnet_Full5pc_1.csv')
sample_DoS = data_DoS.sample(frac=0.1)
data_3ft = pd.read_csv('/content/drive/MyDrive/ML/botiot/UNSW_2018_IoT_Botnet_Full5pc_4.csv')

<ipython-input-3-a3d7fef5f7f2>:1: DtypeWarning: Columns (7,9) have mixed types. Specify dtype option on import or set low_memory=False.
  data_DoS = pd.read_csv('/content/drive/MyDrive/ML/botiot/UNSW_2018_IoT_Botnet_Full5pc_1.csv')
<ipython-input-3-a3d7fef5f7f2>:3: DtypeWarning: Columns (7,9) have mixed types. Specify dtype option on import or set low_memory=False.
  data_3ft = pd.read_csv('/content/drive/MyDrive/ML/botiot/UNSW_2018_IoT_Botnet_Full5pc_4.csv')


In [ ]:
dataset = pd.concat([sample_DoS, data_3ft], ignore_index=True)
dataset

,pkSeqID,stime,flgs,flgs_number,proto,proto_number,saddr,sport,daddr,dport,...,AR_P_Proto_P_DstIP,N_IN_Conn_P_DstIP,N_IN_Conn_P_SrcIP,AR_P_Proto_P_Sport,AR_P_Proto_P_Dport,Pkts_P_State_P_Protocol_P_DestIP,Pkts_P_State_P_Protocol_P_SrcIP,attack,category,subcategory
0,381731,1.528081e+09,e s,2,tcp,1,192.168.100.148,10358,192.168.100.6,80,...,0.236758,78,78,0.262901,0.216460,278,278,1,DoS,TCP
1,252765,1.528081e+09,e,1,tcp,1,192.168.100.150,61647,192.168.100.3,80,...,16.564400,84,84,15.586100,1.312560,252,252,1,DoS,TCP
2,178295,1.528081e+09,e s,2,tcp,1,192.168.100.148,51755,192.168.100.6,80,...,0.254467,7,7,0.254467,0.172913,42,42,1,DoS,TCP
3,349644,1.528081e+09,e s,2,tcp,1,192.168.100.147,10634,192.168.100.7,80,...,0.220205,100,100,0.220205,0.220205,400,400,1,DoS,TCP
4,234592,1.528081e+09,e s,2,tcp,1,192.168.100.149,64859,192.168.100.5,80,...,0.213856,34,34,0.184674,0.559189,54,54,1,DoS,TCP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
768517,3668518,1.529381e+09,e,1,tcp,1,192.168.100.150,35064,192.168.100.3,22,...,9.889330,19,19,455.754000,9.889330,30,30,1,Theft,Keylogging
768518,3668519,1.529381e+09,e,1,tcp,1,192.168.100.150,35066,192.168.100.3,22,...,9.889330,19,19,10453.000000,9.889330,30,30,1,Theft,Keylogging
768519,3668520,1.529381e+09,e,1,tcp,1,192.168.100.150,35070,192.168.100.3,22,...,9.889330,19,19,10.785200,9.889330,441,441,1,Theft,Keylogging
768520,3668521,1.529381e+09,e,1,tcp,1,192.168.100.3,43001,192.168.100.150,4433,...,666667.000000,1,3,666667.000000,22346.400000,2,4,1,Theft,Keylogging


In [ ]:
specific_label = 'DDoS'
n_samples = 100000

# Lọc tập dữ liệu để chỉ chứa các hàng có nhãn cụ thể
filtered_dataset = dataset[dataset['category'] == specific_label]

# Lấy mẫu ngẫu nhiên 500 hàng từ tập dữ liệu đã lọc
sampled_dataset = filtered_dataset.sample(n=n_samples, random_state=42)

# Lọc tập dữ liệu để chỉ chứa các hàng không có nhãn cụ thể
remaining_dataset = dataset[dataset['category'] != specific_label]

# Kết hợp lại tập dữ liệu đã lấy mẫu với phần còn lại của tập dữ liệu
dataset = pd.concat([sampled_dataset, remaining_dataset], ignore_index=True)

# Kiểm tra kích thước của tập dữ liệu cuối cùng
print(dataset.shape)

(291638, 46)


In [ ]:
label_counts = dataset['category'].value_counts()
label_counts

category
DDoS              576884
DoS               100000
Reconnaissance     91082
Normal               477
Theft                 79
Name: count, dtype: int64

In [ ]:
dataset = dataset[dataset['category'] != 'Normal']
dataset = dataset[dataset['category'] != 'Theft']

# Loại bỏ các hàng chứa giá trị NaN
dataset = dataset.dropna()


In [ ]:
label_counts = dataset['category'].value_counts()
label_counts

category
DDoS              100000
DoS               100000
Reconnaissance     91082
Name: count, dtype: int64

In [ ]:
dataset

,pkSeqID,stime,flgs,flgs_number,proto,proto_number,saddr,sport,daddr,dport,...,AR_P_Proto_P_DstIP,N_IN_Conn_P_DstIP,N_IN_Conn_P_SrcIP,AR_P_Proto_P_Sport,AR_P_Proto_P_Dport,Pkts_P_State_P_Protocol_P_DestIP,Pkts_P_State_P_Protocol_P_SrcIP,attack,category,subcategory
0,381731,1.528081e+09,e s,2,tcp,1,192.168.100.148,10358,192.168.100.6,80,...,0.236758,78,78,0.262901,0.216460,278,278,1,DoS,TCP
1,252765,1.528081e+09,e,1,tcp,1,192.168.100.150,61647,192.168.100.3,80,...,16.564400,84,84,15.586100,1.312560,252,252,1,DoS,TCP
2,178295,1.528081e+09,e s,2,tcp,1,192.168.100.148,51755,192.168.100.6,80,...,0.254467,7,7,0.254467,0.172913,42,42,1,DoS,TCP
3,349644,1.528081e+09,e s,2,tcp,1,192.168.100.147,10634,192.168.100.7,80,...,0.220205,100,100,0.220205,0.220205,400,400,1,DoS,TCP
4,234592,1.528081e+09,e s,2,tcp,1,192.168.100.149,64859,192.168.100.5,80,...,0.213856,34,34,0.184674,0.559189,54,54,1,DoS,TCP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
768517,3668518,1.529381e+09,e,1,tcp,1,192.168.100.150,35064,192.168.100.3,22,...,9.889330,19,19,455.754000,9.889330,30,30,1,Theft,Keylogging
768518,3668519,1.529381e+09,e,1,tcp,1,192.168.100.150,35066,192.168.100.3,22,...,9.889330,19,19,10453.000000,9.889330,30,30,1,Theft,Keylogging
768519,3668520,1.529381e+09,e,1,tcp,1,192.168.100.150,35070,192.168.100.3,22,...,9.889330,19,19,10.785200,9.889330,441,441,1,Theft,Keylogging
768520,3668521,1.529381e+09,e,1,tcp,1,192.168.100.3,43001,192.168.100.150,4433,...,666667.000000,1,3,666667.000000,22346.400000,2,4,1,Theft,Keylogging


In [ ]:
def min_max_scaling(data):
  scaled_dataset = (data-data.min())/(data.max()-data.min())
  return scaled_dataset

In [ ]:
# Importing LabelEncoder from Sklearn
# library from preprocessing Module.
from sklearn.preprocessing import LabelEncoder

# Creating a instance of label Encoder.
le = LabelEncoder()

str_columns = dataset.select_dtypes(include=['object']).columns
for col in str_columns:
    dataset[col] = le.fit_transform(dataset[col].astype(str))
# printing label
sample = pd.concat([dataset.head(), dataset.tail()], ignore_index=True)
sample

,pkSeqID,stime,flgs,flgs_number,proto,proto_number,saddr,sport,daddr,dport,...,AR_P_Proto_P_DstIP,N_IN_Conn_P_DstIP,N_IN_Conn_P_SrcIP,AR_P_Proto_P_Sport,AR_P_Proto_P_Dport,Pkts_P_State_P_Protocol_P_DestIP,Pkts_P_State_P_Protocol_P_SrcIP,attack,category,subcategory
0,3112229,1.528099e+09,0,1,4,3,3,46993,16,3663,...,0.490811,100,100,0.493277,0.490811,597,597,1,0,7
1,3127344,1.528099e+09,0,1,4,3,3,54847,16,3663,...,0.575610,100,92,0.575480,0.575610,700,644,1,0,7
2,3005920,1.528099e+09,0,1,4,3,1,62910,16,3663,...,0.872496,100,27,1.025060,0.872496,1195,392,1,0,7
3,3266232,1.528099e+09,0,1,4,3,1,18493,16,3663,...,1.074590,100,100,1.074600,1.074590,1500,1500,1,0,7
4,3190395,1.528099e+09,0,1,4,3,2,36276,16,3663,...,0.752239,100,87,0.723834,0.752239,1045,941,1,0,7
5,3668518,1.529381e+09,0,1,3,1,4,26799,16,1906,...,9.889330,19,19,455.754000,9.889330,30,30,1,4,2
6,3668519,1.529381e+09,0,1,3,1,4,26801,16,1906,...,9.889330,19,19,10453.000000,9.889330,30,30,1,4,2
7,3668520,1.529381e+09,0,1,3,1,4,26806,16,1906,...,9.889330,19,19,10.785200,9.889330,441,441,1,4,2
8,3668521,1.529381e+09,0,1,3,1,6,35464,13,2725,...,666667.000000,1,3,666667.000000,22346.400000,2,4,1,4,2
9,3668522,1.529381e+09,0,1,0,2,6,0,12,0,...,0.018868,2,3,0.018868,0.018868,16,16,1,4,2


In [ ]:
X_df = pd.DataFrame(dataset)


# Tính toán ma trận tương quan
correlation_matrix = X_df.corr()

# Hiển thị tương quan giữa các đặc trưng và nhãn
print(correlation_matrix['category'].sort_values(ascending=False))

category                            1.000000
state                               0.599271
AR_P_Proto_P_DstIP                  0.352654
pkSeqID                             0.258322
AR_P_Proto_P_SrcIP                  0.195407
flgs_number                         0.180932
saddr                               0.178382
AR_P_Proto_P_Dport                  0.170405
AR_P_Proto_P_Sport                  0.158158
rate                                0.153301
flgs                                0.068501
daddr                               0.038755
drate                               0.036042
srate                               0.013600
dpkts                               0.006103
sbytes                              0.004553
bytes                               0.004544
dbytes                              0.003694
pkts                               -0.001782
TnBPSrcIP                          -0.004309
TnBPDstIP                          -0.004322
spkts                              -0.006612
dur       

In [ ]:
# Xác định ngưỡng tương quan
threshold = 0.3

# Lấy danh sách các cột có tương quan tuyệt đối lớn hơn ngưỡng với 'category'
columns_to_remove = correlation_matrix.index[correlation_matrix['category'].abs() > threshold].tolist()

# Loại bỏ nhãn 'category' khỏi danh sách nếu có
columns_to_remove.remove('category')

# Hiển thị các cột sẽ bị loại bỏ
print("Các cột sẽ bị loại bỏ:", columns_to_remove)

# Tạo DataFrame mới với các cột không bị loại bỏ
X_df_reduced = X_df.drop(columns=columns_to_remove)

# Hiển thị DataFrame mới
print(X_df_reduced)

Các cột sẽ bị loại bỏ: ['stime', 'proto', 'proto_number', 'dport', 'state', 'state_number', 'ltime', 'seq', 'mean', 'stddev', 'min', 'max', 'AR_P_Proto_P_DstIP', 'Pkts_P_State_P_Protocol_P_DestIP', 'Pkts_P_State_P_Protocol_P_SrcIP', 'subcategory']
        pkSeqID  flgs  flgs_number  saddr  sport  daddr  pkts  bytes  \
0       3112229     0            1      3  46993     16     6    360   
1       3127344     0            1      3  54847     16     7    420   
2       3005920     0            1      1  62910     16    14    840   
3       3266232     0            1      1  18493     16    15    900   
4       3190395     0            1      2  36276     16    10    600   
...         ...   ...          ...    ...    ...    ...   ...    ...   
291633  3668518     0            1      4  26799     16     6    434   
291634  3668519     0            1      4  26801     16     6    434   
291635  3668520     0            1      4  26806     16    31   5472   
291636  3668521     0           

In [ ]:
dataset_reduced = dataset.drop(columns=columns_to_remove)

In [ ]:
dataset_reduced

,pkSeqID,flgs,flgs_number,saddr,sport,daddr,pkts,bytes,dur,sum,...,TnP_PDstIP,TnP_PerProto,TnP_Per_Dport,AR_P_Proto_P_SrcIP,N_IN_Conn_P_DstIP,N_IN_Conn_P_SrcIP,AR_P_Proto_P_Sport,AR_P_Proto_P_Dport,attack,category
0,3112229,0,1,3,46993,16,6,360,12.163544,8.120798,...,597,597,597,0.490811,100,100,0.493277,0.490811,1,0
1,3127344,0,1,3,54847,16,7,420,12.163751,8.101995,...,700,700,700,0.575480,100,92,0.575480,0.575610,1,0
2,3005920,0,1,1,62910,16,14,840,13.657685,10.749609,...,1195,1195,1195,1.063030,100,27,1.025060,0.872496,1,0
3,3266232,0,1,1,18493,16,15,900,13.958619,11.960337,...,1500,1500,1500,1.074590,100,100,1.074600,1.074590,1,0
4,3190395,0,1,2,36276,16,10,600,13.815331,11.074812,...,1045,1045,1045,0.783010,100,87,0.723834,0.752239,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291633,3668518,0,1,4,26799,16,6,434,0.013165,0.013165,...,471,475,471,9.889330,19,19,455.754000,9.889330,1,4
291634,3668519,0,1,4,26801,16,6,434,0.000574,0.000574,...,471,475,471,9.889330,19,19,10453.000000,9.889330,1,4
291635,3668520,0,1,4,26806,16,31,5472,2.874302,2.874302,...,471,475,471,9.889330,19,19,10.785200,9.889330,1,4
291636,3668521,0,1,6,35464,13,2,134,0.000003,0.000003,...,2,475,4,22346.400000,1,3,666667.000000,22346.400000,1,4


In [ ]:
dataset_reduced = dataset_reduced.drop(columns='attack')

In [ ]:
label_column = 'category'
scaled_dataset = dataset_reduced.copy()

# Tách cột nhãn ra khỏi tập dữ liệu
labels = scaled_dataset[label_column]
scaled_dataset = scaled_dataset.drop(columns=[label_column])

scaled_dataset = min_max_scaling(scaled_dataset)


In [ ]:
scaled_dataset

,pkSeqID,flgs,flgs_number,saddr,sport,daddr,pkts,bytes,dur,sum,...,TnP_PSrcIP,TnP_PDstIP,TnP_PerProto,TnP_Per_Dport,AR_P_Proto_P_SrcIP,N_IN_Conn_P_DstIP,N_IN_Conn_P_SrcIP,AR_P_Proto_P_Sport,AR_P_Proto_P_Dport,attack
0,0.848360,0.0,0.0,0.15,0.739197,0.192771,0.000071,0.000004,4.388818e-03,4.244629e-03,...,0.002643,0.002643,0.002610,0.002438,1.808248e-07,1.000000,1.000000,1.644257e-07,2.454055e-07,1.0
1,0.852480,0.0,0.0,0.15,0.862740,0.192771,0.000086,0.000005,4.388893e-03,4.234801e-03,...,0.002851,0.003100,0.003061,0.002860,2.120186e-07,1.000000,0.919192,1.918267e-07,2.878050e-07,1.0
2,0.819381,0.0,0.0,0.05,0.989571,0.192771,0.000186,0.000011,4.927930e-03,5.618673e-03,...,0.001734,0.005294,0.005228,0.004885,3.916420e-07,1.000000,0.262626,3.416867e-07,4.362480e-07,1.0
3,0.890340,0.0,0.0,0.05,0.290894,0.192771,0.000200,0.000012,5.036512e-03,6.251504e-03,...,0.006647,0.006647,0.006564,0.006133,3.959010e-07,1.000000,1.000000,3.582000e-07,5.372950e-07,1.0
4,0.869667,0.0,0.0,0.10,0.570620,0.192771,0.000128,0.000008,4.984812e-03,5.788652e-03,...,0.004168,0.004629,0.004571,0.004271,2.884769e-07,1.000000,0.868687,2.412780e-07,3.761195e-07,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291633,0.999999,0.0,0.0,0.20,0.421547,0.192771,0.000071,0.000005,4.750161e-06,6.881165e-06,...,0.002084,0.002084,0.002076,0.001923,3.643432e-06,0.181818,0.181818,1.519180e-04,4.944665e-06,1.0
291634,0.999999,0.0,0.0,0.20,0.421578,0.192771,0.000071,0.000005,2.071092e-07,3.000219e-07,...,0.002084,0.002084,0.002076,0.001923,3.643432e-06,0.181818,0.181818,3.484333e-03,4.944665e-06,1.0
291635,0.999999,0.0,0.0,0.20,0.421657,0.192771,0.000428,0.000075,1.037098e-03,1.502358e-03,...,0.002084,0.002084,0.002076,0.001923,3.643432e-06,0.181818,0.181818,3.595067e-06,4.944665e-06,1.0
291636,1.000000,0.0,0.0,0.30,0.557847,0.156627,0.000014,0.000001,1.082452e-09,1.568059e-09,...,0.000084,0.000004,0.002076,0.000012,8.232871e-03,0.000000,0.020202,2.222223e-01,1.117320e-02,1.0


In [ ]:
y = labels
y.unique()

array([0, 1, 2, 3, 4])

In [ ]:
correlation_matrix = dataset_reduced.corr()
print(correlation_matrix['category'].sort_values(ascending=False))


category              1.000000
pkSeqID               0.258322
AR_P_Proto_P_SrcIP    0.195407
flgs_number           0.180932
saddr                 0.178382
AR_P_Proto_P_Dport    0.170405
AR_P_Proto_P_Sport    0.158158
rate                  0.153301
flgs                  0.068501
daddr                 0.038755
drate                 0.036042
srate                 0.013600
dpkts                 0.006103
sbytes                0.004553
bytes                 0.004544
dbytes                0.003694
pkts                 -0.001782
TnBPSrcIP            -0.004309
TnBPDstIP            -0.004322
spkts                -0.006612
dur                  -0.017504
attack               -0.023536
sport                -0.026195
TnP_PerProto         -0.042788
TnP_PSrcIP           -0.166362
TnP_PDstIP           -0.171855
TnP_Per_Dport        -0.185852
sum                  -0.207175
N_IN_Conn_P_DstIP    -0.250839
N_IN_Conn_P_SrcIP    -0.296739
Name: category, dtype: float64


In [ ]:
from sklearn.model_selection import train_test_split



# Tách thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(scaled_dataset, y, test_size=0.2, random_state=42)

# In kích thước của các tập dữ liệu
print("Kích thước tập huấn luyện:", X_train.shape)
print("Kích thước tập kiểm tra:", X_test.shape)


Kích thước tập huấn luyện: (233310, 29)
Kích thước tập kiểm tra: (58328, 29)


In [ ]:
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold, cross_val_score
from xgboost import XGBClassifier

models = [svm.SVC(kernel='rbf', C=100, gamma='auto',class_weight='balanced' ,probability=True),
          XGBClassifier(learning_rate=0.01, n_estimators=100, max_depth=1, gamma=10, subsample=0.5, colsample_bytree=0.5),
          KNeighborsClassifier(n_neighbors=20, weights='uniform', algorithm='ball_tree', p=1, metric='minkowski')]

classifiers = ['SVM',
               'XGB',
               'KNeighbors']



In [ ]:


def acc_score():
    Score = pd.DataFrame({"Classifier":classifiers})
    j = 0
    acc = []
    recall = []
    for i in models:
        model = i
        model.fit(X_train,y_train)
        predictions = model.predict(X_test)
        acc.append(accuracy_score(y_test,predictions))
        recall.append(metrics.recall_score(y_test, predictions, average="macro"))
        j = j+1
    Score["Accuracy"] = acc
    Score["Recall"] = recall
    Score.sort_values(by="Accuracy", ascending=False,inplace = True)
    Score.reset_index(drop=True, inplace=True)
    return Score


In [7]:
score1 = acc_score()
score1

   Classifier  Accuracy   Recall
0         SVM  0.811274 0.811274
1         XGB  0.823371 0.823371
2  KNeighbors  0.851892 0.851892
